<a href="https://colab.research.google.com/github/anafrnandes/etl-worldbank/blob/main/Semana4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install jupyter-dash dash plotly pandas

from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import sqlite3

# Conectar à base de dados
conn = sqlite3.connect('/content/etl_worldbank.db')
df = pd.read_sql_query("SELECT * FROM indicadores_pais", conn)

#  app
app = JupyterDash(__name__)

# Gráfico de dispersão PIB per Capita vs População
scatter_fig = px.scatter(df,
                         x='Population_2022',
                         y='gdp_per_capita_2022',
                         title='PIB per Capita vs População (2022)',
                         size='Population_2022',
                         hover_name='Country Name')

df_map = df[df['Country Code'].str.len() == 3]

# Mapa PIB per Capita por país
map_fig = px.choropleth(df_map,
                        locations="Country Code",
                        color="gdp_per_capita_2022",
                        hover_name="Country Name",
                        color_continuous_scale="Viridis",
                        range_color=[0, 80000],
                        projection="natural earth",
                        title="PIB per Capita por País (2022)")

map_fig.update_geos(
    showframe=False,
    showcoastlines=True,
    projection_type="natural earth"
)


# Layout do app
app.layout = html.Div([
    html.H1("Painel de Indicadores País (World Bank)", style={'textAlign': 'center'}),

    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': c, 'value': c} for c in df['Country Name'].unique()],
        value='Brazil'
    ),

    dcc.Graph(id='gdp-line'),
    html.Br(),
    dcc.Graph(id='population-bar'),
    html.Br(),

    dcc.Graph(id='scatter-graph', figure=scatter_fig),
    html.Br(),
    dcc.Graph(id='map-graph', figure=map_fig)
])

# gráficos interativos
@app.callback(
    [Output('gdp-line', 'figure'),
     Output('population-bar', 'figure')],
    [Input('country-dropdown', 'value')]
)
def update_country_graphs(selected_country):
    filtered_df = df[df['Country Name'] == selected_country]

    gdp_fig = px.line(filtered_df.melt(id_vars=['Country Name'], value_vars=[
        'gdp_per_capita_2018', 'gdp_per_capita_2019', 'gdp_per_capita_2020',
        'gdp_per_capita_2021', 'gdp_per_capita_2022'
    ]),
        x='variable', y='value',
        title=f'Evolução do PIB per Capita ({selected_country})')

    pop_fig = px.bar(filtered_df.melt(id_vars=['Country Name'], value_vars=[
        'Population_2018', 'Population_2019', 'Population_2020',
        'Population_2021', 'Population_2022'
    ]),
        x='variable', y='value',
        title=f'População ({selected_country})')

    return gdp_fig, pop_fig

# Rodar no Colab
app.run(mode='inline')



/usr/local/lib/python3.11/dist-packages/dash/dash.py:587: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>